Get

In [1]:
 /* 
From Windows Internals book by David Solomon and Mark Russinovich:
"The default level of available memory that signals a low-memory-resource notification event is approximately 32 MB per 4 GB, 
to a maximum of 64 MB. The default level that signals a high-memory-resource notification event is three times the default low-memory value."
*/ 

SET NOCOUNT ON;
SET ANSI_WARNINGS ON;
SET QUOTED_IDENTIFIER ON;
SET DATEFORMAT mdy;

DECLARE @masterpid int, @RegKey NVARCHAR(255), @LowMemoryThreshold int, @systemmem bigint
DECLARE @permstbl TABLE ([name] sysname);

DECLARE @sqlcmd NVARCHAR(max), @params NVARCHAR(600)
DECLARE @sqlmajorver int
DECLARE @ErrorMessage NVARCHAR(4000)

SELECT @sqlmajorver = CONVERT(int, (@@microsoftversion / 0x1000000) & 0xff);

SELECT @masterpid = principal_id FROM master.sys.database_principals (NOLOCK) WHERE sid = SUSER_SID()

INSERT INTO @permstbl
SELECT a.name
FROM master.sys.all_objects a (NOLOCK) INNER JOIN master.sys.database_permissions b (NOLOCK) ON a.[OBJECT_ID] = b.major_id
WHERE a.type IN ('P', 'X') AND b.grantee_principal_id <>0
AND b.grantee_principal_id <>2
AND b.grantee_principal_id = @masterpid;

IF @sqlmajorver = 9
BEGIN
	SET @sqlcmd = N'SELECT @systemmemOUT = t1.record.value(''(./Record/MemoryRecord/TotalPhysicalMemory)[1]'', ''bigint'')/1024
FROM (SELECT MAX([TIMESTAMP]) AS [TIMESTAMP], CONVERT(xml, record) AS record 
	FROM sys.dm_os_ring_buffers (NOLOCK)
	WHERE ring_buffer_type = N''RING_BUFFER_RESOURCE_MONITOR''
		AND record LIKE ''%RESOURCE_MEMPHYSICAL%''
	GROUP BY record) AS t1';
END
ELSE
BEGIN
	SET @sqlcmd = N'SELECT @systemmemOUT = total_physical_memory_kb/1024 FROM sys.dm_os_sys_memory';
END

SET @params = N'@systemmemOUT bigint OUTPUT';

EXECUTE sp_executesql @sqlcmd, @params, @systemmemOUT=@systemmem OUTPUT;

IF (ISNULL(IS_SRVROLEMEMBER(N'sysadmin'), 0) = 1) OR ((SELECT COUNT([name]) FROM @permstbl WHERE [name] = 'xp_regread') = 1)
BEGIN
	BEGIN TRY
		SELECT @RegKey = N'System\CurrentControlSet\Control\SessionManager\MemoryManagement'
		EXEC master.sys.xp_regread N'HKEY_LOCAL_MACHINE', @RegKey, N'LowMemoryThreshold', @LowMemoryThreshold OUTPUT, NO_OUTPUT
		
		IF @LowMemoryThreshold IS NULL
		SELECT @LowMemoryThreshold = CASE WHEN @systemmem <= 4096 THEN 32 ELSE 64 END
	END TRY
	BEGIN CATCH
		SELECT ERROR_NUMBER() AS ErrorNumber, ERROR_MESSAGE() AS ErrorMessage;
		SELECT @ErrorMessage = 'Server Memory subsection - Error raised in TRY block. ' + ERROR_MESSAGE()
		RAISERROR (@ErrorMessage, 16, 1);
	END CATCH
END
ELSE
BEGIN
	RAISERROR('WARNING: Missing permissions for full "Instance info" checks. Bypassing LowMemoryThreshold check', 16, 1, N'sysadmin')
	--RETURN
END;

IF @LowMemoryThreshold IS NOT NULL
SELECT 'Memory_checks' AS [Category], 'Memory_RM_Tresholds' AS [Information], @LowMemoryThreshold AS [MEMPHYSICAL_LOW_Threshold], @LowMemoryThreshold * 3 AS [MEMPHYSICAL_HIGH_Threshold]

SELECT 'Memory_checks' AS [Category], 'Memory_RM_Notifications' AS [Information], 
CASE WHEN x.[TIMESTAMP] BETWEEN -2147483648 AND 2147483647 AND si.ms_ticks BETWEEN -2147483648 AND 2147483647 THEN DATEADD(ms, x.[TIMESTAMP] - si.ms_ticks, GETDATE()) 
    ELSE DATEADD(s, ([TIMESTAMP]/1000) - (si.ms_ticks/1000), GETDATE()) END AS Event_Time,
    record.value('(./Record/ResourceMonitor/Notification)[1]', 'VARCHAR(max)') AS [Notification],
    record.value('(./Record/MemoryRecord/TotalPhysicalMemory)[1]', 'bigint')/1024 AS [Total_Physical_Mem_MB],
    record.value('(./Record/MemoryRecord/AvailablePhysicalMemory)[1]', 'bigint')/1024 AS [Avail_Physical_Mem_MB],
    record.value('(./Record/MemoryRecord/AvailableVirtualAddressSpace)[1]', 'bigint')/1024 AS [Avail_VAS_MB],
    record.value('(./Record/MemoryRecord/TotalPageFile)[1]', 'bigint')/1024 AS [Total_Pagefile_MB],
    record.value('(./Record/MemoryRecord/AvailablePageFile)[1]', 'bigint')/1024 AS [Avail_Pagefile_MB]
FROM (SELECT [TIMESTAMP], CONVERT(xml, record) AS record 
            FROM sys.dm_os_ring_buffers (NOLOCK)
            WHERE ring_buffer_type = N'RING_BUFFER_RESOURCE_MONITOR') AS x
CROSS JOIN sys.dm_os_sys_info si (NOLOCK)
--WHERE CASE WHEN x.[timestamp] BETWEEN -2147483648 AND 2147483648 THEN DATEADD(ms, x.[timestamp] - si.ms_ticks, GETDATE()) 
--	ELSE DATEADD(s, (x.[timestamp]/1000) - (si.ms_ticks/1000), GETDATE()) END >= DATEADD(hh, -12, GETDATE())
ORDER BY 2 DESC;

Commands completed successfully.

Total execution time: 00:00:00.208

Category,Information,MEMPHYSICAL_LOW_Threshold,MEMPHYSICAL_HIGH_Threshold
Memory_checks,Memory_RM_Tresholds,64,192


Category,Information,Event_Time,Notification,Total_Physical_Mem_MB,Avail_Physical_Mem_MB,Avail_VAS_MB,Total_Pagefile_MB,Avail_Pagefile_MB
Memory_checks,Memory_RM_Notifications,2021-10-18 16:53:32.203,RESOURCE_MEMPHYSICAL_HIGH,16301,2557,134175808,33709,15656
Memory_checks,Memory_RM_Notifications,2021-10-18 16:53:32.203,RESOURCE_MEM_STEADY,16301,2557,134175808,33709,15656
Memory_checks,Memory_RM_Notifications,2021-10-18 16:53:26.453,RESOURCE_MEMPHYSICAL_LOW,16301,2570,134175808,33709,15685
Memory_checks,Memory_RM_Notifications,2021-10-18 16:52:25.877,RESOURCE_MEM_STEADY,16301,2624,134175806,33709,15786
Memory_checks,Memory_RM_Notifications,2021-10-18 16:52:21.777,RESOURCE_MEMPHYSICAL_LOW,16301,2622,134175806,33709,15786
Memory_checks,Memory_RM_Notifications,2021-10-18 16:51:20.207,RESOURCE_MEMPHYSICAL_HIGH,16301,2618,134175804,33709,15798
Memory_checks,Memory_RM_Notifications,2021-10-18 16:51:20.207,RESOURCE_MEM_STEADY,16301,2618,134175804,33709,15798
Memory_checks,Memory_RM_Notifications,2021-10-18 16:51:15.693,RESOURCE_MEMPHYSICAL_LOW,16301,2611,134175804,33709,15708
Memory_checks,Memory_RM_Notifications,2021-10-18 16:50:14.140,RESOURCE_MEMPHYSICAL_HIGH,16301,2548,134175804,33709,15718
Memory_checks,Memory_RM_Notifications,2021-10-18 16:50:14.137,RESOURCE_MEM_STEADY,16301,2548,134175804,33709,15718
